In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("afg.csv",sep=";")

In [3]:
data.head(3) # just to make sure the data set was well imported

,OBJECTID,SOURCE,VIL_UID,CENTER,CNTR_CODE,AFG_UID,POPULATION,LANGUAGE_,LANG_CODE,ELEVATION,...,LON_X,MISTI_PROV,MISTI_DIST,VILLAGE_NA,DIST_CODE,DIST_NA_EN,DIST_NA_DA,PROV_NA_EN,PROV_NA_DA,PROV_CODE
0,1,CSO AIMS Villages (ACSOR),QLT-113,NaN,0,ZBL-QLT-113,0,Pashto,2,1716.072021,...,66.71265,ZBL,QLT,Enjergay,2401,Qalat,????,Zabul,????,24
1,2,USAID AIDVillageView,MZI-016,NaN,0,KST-MZI-016,0,Pashto,2,1339.345947,...,69.74210,KST,MZI,Nekbal,2605,Mandozayi,????????,Khost,????,26
2,3,USAID AIDVillageView,MZI-005,NaN,0,KST-MZI-005,0,Pashto,2,1314.277954,...,69.77490,KST,MZI,Bar Khankhel,2605,Mandozayi,????????,Khost,????,26


# adding colums to the dataset

In [4]:
def f(data):
    if data['POPULATION']>1000000:
        val = data['POPULATION']/1000
    elif data['POPULATION'] > 100000:
        val = data['POPULATION']/700
    elif data['POPULATION'] > 10000:
        val = data['POPULATION']/300
    elif data['POPULATION'] > 1000:
        val = data['POPULATION']/80
    elif data['POPULATION'] > 100:
        val = data['POPULATION']/90
    elif data['POPULATION'] > 10:
        val = data['POPULATION']/9.99
    else:
        val = 1
    return val

data["weight_pop"]=data.apply(f, axis=1)

In [5]:
def g(data):
    if data['POPULATION']>1000000:
        val = "#b8860b"
    elif data['POPULATION'] > 100000:
        val = "#ffd700"
    elif data['POPULATION'] > 10000:
        val = "#eedd82"
    elif data['POPULATION'] > 1000:
        val = "#eee8aa"
    elif data['POPULATION'] > 100:
        val = "#ffffe0"
    elif data['POPULATION'] > 10:
        val = "#ffffe0"
    else:
        val = "green"
    return val

data["weight_pop_col"]=data.apply(g, axis=1)

In [6]:
data["weight_long"]=data.LON_X*data.POPULATION
data["weight_lat"]=data.LAT_Y*data.POPULATION

In [7]:
groups=data.groupby(["PROV_CODE"])
region=pd.DataFrame(columns=['PROV_CODE','POP','LANG_CODE','LON_X','LAT_Y','weight_long','weight_lat'])
i=0
for key,df in groups:
    region.loc[i]=[key,df['POPULATION'].sum(),df["LANG_CODE"].value_counts().idxmax(),df["LON_X"].mean(),df['LAT_Y'].mean(),
                  df['weight_long'].sum()/df['POPULATION'].sum(),df['weight_lat'].sum()/df['POPULATION'].sum()]
    i=i+1

In [8]:
groups=data.groupby(["DIST_CODE"])
district=pd.DataFrame(columns=['DIST_CODE','POP',"Number_centroids","PEOPLE_per_centroids",'LANG_CODE','LON_X','LAT_Y','weight_long','weight_lat'])
i=0
for key,df in groups:
    district.loc[i]=[key,df['POPULATION'].sum(),df['DIST_CODE'].count(),df['POPULATION'].sum()/df["DIST_CODE"].count(),df["LANG_CODE"].value_counts().idxmax(),df["LON_X"].mean(),df['LAT_Y'].mean(),
                  df['weight_long'].sum()/(df['POPULATION'].sum()+1),df['weight_lat'].sum()/(df['POPULATION'].sum()+1)]
    i=i+1

In [9]:
def find_city(longitude, latitude, big_town):
    closest = 0
    centroid = big_town[["LON_X","LAT_Y"]].iloc[0]
    min_dis = (longitude - centroid[0])**2 + (latitude - centroid[1])**2 
    for i in range(1, big_town[["LON_X","LAT_Y"]].shape[0]):
        dis = (longitude - big_town[["LON_X","LAT_Y"]].iloc[i][0])**2 + (latitude - big_town[["LON_X","LAT_Y"]].iloc[i][1])**2 
        if dis < min_dis:
            min_dis = dis
            closest = i
            centroid = big_town[["LON_X","LAT_Y"]].iloc[i]
    return big_town["OBJECTID"].iloc[closest], centroid

In [10]:
groups=data.groupby(["DIST_CODE"])
district=pd.DataFrame(columns=['DIST_CODE','POP',"Number_centroids","PEOPLE_per_centroids",'LANG_CODE','LON_X','LAT_Y','weight_long','weight_lat'])
i=0
for key,df in groups:
    district.loc[i]=[key,df['POPULATION'].sum(),df['DIST_CODE'].count(),df['POPULATION'].sum()/df["DIST_CODE"].count(),df["LANG_CODE"].value_counts().idxmax(),df["LON_X"].mean(),df['LAT_Y'].mean(),
                  df['weight_long'].sum()/(df['POPULATION'].sum()+1),df['weight_lat'].sum()/(df['POPULATION'].sum()+1)]
    i=i+1

In [11]:
#the 399 districts
list_dist=district[district["DIST_CODE"]!=0]["DIST_CODE"].tolist()
list_dist=set(list_dist)
list_dist;

# this is the part which is going to take a lot of time to complete the calculation

In [34]:
a=999 #all the settlement with less than 999 people are going to be merged with settlement from the same district 
#that have more than 1000 people. You can toy with these parameters if it takes to long (ie : a=50 and b=10000)
b=1000

In [32]:
# Find out which small town is close to a given big town 
data_small=data[data["POPULATION"]<a].sample(frac=1, replace=False);
data_big=data[data["POPULATION"]>b].sample(frac=1, replace=False);
new_small_data=pd.DataFrame(columns=["OBJECTID","which_big_town"])

k=0
for j in list_dist:
    length=data_small[data_small["DIST_CODE"]==j].shape[0]
    location_data = data_small[data_small["DIST_CODE"]==j][["LON_X","LAT_Y"]].as_matrix()
    #which_centroid = np.zeros(data_small[data_small["DIST_CODE"]==j].shape[0])
    if data_small[(data_small["DIST_CODE"]==j)].shape[0]!=0 and data_big[(data_big["DIST_CODE"]==j)].shape[0]!=0:
        for i in range(0,length):
            town, dummy = find_city(location_data[i][0], location_data[i][1], data_big[data_big["DIST_CODE"]==j])
            #new_small_data.loc[k]=[data_small["OBJECTID"].iloc[i], town]
            new_small_data.loc[k]=[data_small[data_small["DIST_CODE"]==j]["OBJECTID"].iloc[i], town]
            k=k+1
            #which_centroid[i], dummy = find_city(location_data[i][0], location_data[i][1], data_big[data_big["DIST_CODE"]==j][["LON_X","LAT_Y"]])
        #new_small_data[new_small_data]['which_big_town'] = which_centroid    #which town is it linked to

In [33]:
#check the result
print(new_small_data.shape[0],data_small.shape[0])


258 7528


In [16]:
new_data=pd.merge(new_small_data,data,on="OBJECTID",how="left")[["OBJECTID","which_big_town","POPULATION","LANGUAGE_","LANG_CODE","LON_X","LAT_Y"]]
new_data["weight_pop"]=new_data.apply(f, axis=1)
new_data["weight_pop_col"]=new_data.apply(g, axis=1)
new_data.shape[0]

258

In [17]:
# just to make sure everything is good so far
new_data.head(5)

,OBJECTID,which_big_town,POPULATION,LANGUAGE_,LANG_CODE,LON_X,LAT_Y,weight_pop,weight_pop_col
0,37056.0,44832.0,0,Dari,1,69.0354,34.5011,1,green
1,44805.0,44832.0,0,Dari,1,69.0881,34.4843,1,green
2,44827.0,44832.0,0,Dari,1,69.1606,34.4920,1,green
3,44308.0,44832.0,0,Dari,1,69.1441,34.5379,1,green
4,44286.0,44832.0,0,Dari,1,69.0765,34.4729,1,green


In [18]:
new_data["isna"]=new_data["which_big_town"].apply(lambda x : pd.isnull(x))
new_data["which_big_town"].fillna(value=0,inplace=True)
new_data["new_town"]=new_data["isna"]*new_data["OBJECTID"]+new_data["which_big_town"]

In [19]:
print(len(new_data["POPULATION"]))

258


In [20]:
# groupby which_big_town,
# check the language spoken on this big town, add the population, keep the longitude, the latitude
groups=new_data.groupby(["which_big_town"])
bigtown=pd.DataFrame(columns=["which_big_town",'POP','LANG_CODE','LON_X','LAT_Y'])
i=0
for key,df in groups:
    bigtown.loc[i]=[key,df['POPULATION'].sum(),df["LANG_CODE"].value_counts().idxmax(),df["LON_X"].mean(),df['LAT_Y'].mean()]
    i=i+1

In [21]:
# making sure everything is good
bigtown.head(3)

,which_big_town,POP,LANG_CODE,LON_X,LAT_Y
0,9181.0,8.0,2.0,65.704030,31.581593
1,27885.0,0.0,2.0,70.457550,34.415350
2,31463.0,6.0,1.0,68.870061,36.733148


In [22]:
data_big["which_big_town"]=data_big["OBJECTID"]

In [23]:
bigtown_merged=pd.merge(data_big,bigtown,on="which_big_town",how="right")[["OBJECTID","which_big_town","POPULATION","POP","LANGUAGE_","LON_X_y","LAT_Y_y"]]#[["OBJECTID","which_big_town","POPULATION","LANGUAGE_","LANG_CODE","LON_X","LAT_Y"]]

In [24]:
bigtown_merged["pop_total"]=bigtown_merged["POPULATION"]+bigtown_merged["POP"]

In [25]:
# making sure it is good
bigtown_merged.head(2)

,OBJECTID,which_big_town,POPULATION,POP,LANGUAGE_,LON_X_y,LAT_Y_y,pop_total
0,44841,44841,300600,0.0,Hazaragi,67.10080,36.686500,300600.0
1,9181,9181,324800,8.0,Pashto,65.70403,31.581593,324808.0


# tools to plot


In [26]:
def f(data):
    if data['pop_total']>1000000:
        val = data['pop_total']/1000
    elif data['pop_total'] > 100000:
        val = data['pop_total']/700
    elif data['pop_total'] > 10000:
        val = data['pop_total']/300
    elif data['pop_total'] > 1000:
        val = data['pop_total']/80
    elif data['pop_total'] > 100:
        val = data['pop_total']/90
    elif data['pop_total'] > 10:
        val = data['pop_total']/9.99
    else:
        val = 1
    return val


In [27]:
def g(data):
    if data['pop_total']>1000000:
        val = "#b8860b"
    elif data['pop_total'] > 100000:
        val = "#ffd700"
    elif data['pop_total'] > 10000:
        val = "#eedd82"
    elif data['pop_total'] > 1000:
        val = "#eee8aa"
    elif data['pop_total'] > 100:
        val = "#ffffe0"
    elif data['pop_total'] > 10:
        val = "#ffffe0"
    else:
        val = "green"
    return val

In [28]:
bigtown_merged["weight_pop"]=bigtown_merged.apply(f, axis=1)

In [29]:
bigtown_merged["weight_pop_col"]=bigtown_merged.apply(g, axis=1)